In [ ]:
import pandas as pd
from pathlib import Path
import shutil
from tqdm import tqdm
import cv2
import numpy as np
import os
import random
#from prettytable import PrettyTable

import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from tqdm import tqdm

from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import pickle, shutil, random

In [ ]:
BASE_PATH = Path('/project/volume/data/in/EmoReact')
LABELS_PATH = Path('/project/volume/data/in/EmoReact/EmoReact_V_1.0/Labels/')
ORIG_FOLDERS = ['Test', 'Train', 'Validation']
LABEL_FILES = ['test_labels.text', 'train_labels.text/', 'val_labels.text']

labels = ['Curiosity', 'Uncertainty', 'Excitement', 'Happiness', 'Surprise', 'Disgust', 'Fear', 'Frustration']

In [ ]:
def create_folders():
    for label in labels:
        label_path = BASE_PATH / Path(label)
        if not label_path.exists():
            label_path.mkdir(parents=True, exist_ok=True)

def read_labels():
    df_train, df_test, df_val = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for label in LABEL_FILES:
        path = LABELS_PATH / Path(label)
        df = pd.read_csv(str(path), header=None)
        df = df.drop(columns=[df.columns[-1]])
        df.columns = labels
        if 'test_labels.text' in label:
            df_train = pd.concat([df_train, df], ignore_index=True)
        elif 'train_labels.text' in label:
            df_test = pd.concat([df_test, df], ignore_index=True)
        elif 'val_labels.text' in label:
            df_val = pd.concat([df_val, df], ignore_index=True)

    return df_train, df_test, df_val

In [ ]:
create_folders()

In [ ]:
df_train, df_test, df_val = read_labels()

In [ ]:
def create_sets():
    for folder in tqdm(ORIG_FOLDERS):
        folder_path = BASE_PATH / Path('EmoReact_V_1.0/Data') / folder
        if 'Test' in folder:
            df = df_test
        elif 'Train' in folder:
            df = df_train
        elif 'Validation' in folder:
            df = df_val
        else:
            continue

        for file_idx, file in enumerate(list(folder_path.rglob('*.mp4'))):
            if file_idx >= len(df):
                break
            file_label = df.loc[file_idx].idxmax() 
            destination_folder = BASE_PATH / Path(file_label)
            destination_folder.mkdir(exist_ok=True, parents=True)
            shutil.move(str(file), str(destination_folder))
        
        if folder_path.exists() and folder_path.is_dir():
            shutil.rmtree(folder_path)

In [ ]:
move_files()

In [ ]:
def get_video_lenght():
    video_lengths = []
    for file in tqdm(list(BASE_PATH.rglob('*.mp4'))):
        cap = cv2.VideoCapture(str(file))
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = frame_count / fps
        video_lengths.append(duration)
        cap.release()
    return video_lengths

In [ ]:
video_lengths = get_video_lenght()

In [ ]:
average_length = np.mean(video_lengths)
std_dev = np.std(video_lengths)

print(average_length, std_dev)

In [ ]:
def extract_frames():
    for p in tqdm(list(BASE_PATH.rglob('*.mp4'))):
        cap = cv2.VideoCapture(str(p))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        nbr_frames = int(total_frames * 0.25)

        for frame_idx in range(nbr_frames):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            _, frame = cap.read()
            frame_file_name = Path(str(p.stem) + f'_{frame_idx}.jpg')
            frame_path = str(p.parent / frame_file_name)
            if not os.path.exists(str(frame_path)):
                cv2.imwrite(frame_path, frame)
    
        cap.release()
        if os.path.exists(str(p)):
            os.remove(str(p))

In [ ]:
extract_frames()

In [ ]:
BASE_PATH = Path('/project/volume/data/out/EmoReact')

IMAGE_PATHS = list(BASE_PATH.rglob('*.jpg'))
IMAGE_PATHS_STR = [str(path) for path in IMAGE_PATHS]

In [ ]:
from mediapipe import Image
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [ ]:
model_path_detect = '/project/volume/models/blaze_face_short_range.tflite'
model_path_mesh = '/project/volume/models/face_landmarker.task'

base_options_detect = python.BaseOptions(model_asset_path=model_path_detect)
options_detect = vision.FaceDetectorOptions(base_options=base_options_detect)
detector_detect = vision.FaceDetector.create_from_options(options_detect)

base_options_mesh = python.BaseOptions(model_asset_path=model_path_mesh)
options_mesh = vision.FaceLandmarkerOptions(base_options=base_options_mesh,
                                       output_face_blendshapes=False,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector_mesh = vision.FaceLandmarker.create_from_options(options_mesh)

In [ ]:
train_prop = 0.6
test_prop = 0.2
valid_prop = 0.2

number_of_images = len(list(BASE_PATH.rglob('*.jpg')))

print("[INFO] Number of images in total ..." + str(number_of_images))

n_train = int((number_of_images * train_prop) + 0.5)
n_valid = int((number_of_images * valid_prop) + 0.5)
n_test = number_of_images - n_train - n_valid


print(f"[INFO] Number of images used in training ... {str(n_train)} ({str(train_prop * 100)}%)")
print(f"[INFO] Number of images used in testing ... {str(n_test)} ({str(test_prop * 100)}%)")
print(f"[INFO] Number of images used in validation ...{str(n_valid)} ({str(valid_prop * 100)}%)")

In [ ]:
def split_and_move():
    for label in tqdm(labels):
        folder_path = BASE_PATH / label
        train_destination = BASE_PATH / "train" / label
        val_destination = BASE_PATH / "val" / label
        test_destination = BASE_PATH / "test" / label

        train_destination.mkdir(parents=True, exist_ok=True)
        val_destination.mkdir(parents=True, exist_ok=True)
        test_destination.mkdir(parents=True, exist_ok=True)

        files = list(folder_path.rglob('*.jpg'))
        random.shuffle(files)
        train_n = (int((len(files) * train_prop) + 0.5))
        val_n = (int((len(files) * valid_prop) + 0.2))
        
        for file_idx, file in enumerate(list(folder_path.rglob('*.jpg'))):
            if file_idx < train_n:
                shutil.move(str(file), train_destination)
            elif file_idx < train_n + val_n:
                shutil.move(str(file), val_destination)
            else:
                shutil.move(str(file), test_destination)

        if folder_path.exists() and folder_path.is_dir():
            shutil.rmtree(folder_path)

In [ ]:
split_and_move()

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

def get_distribution():
    nbr_per_cat = Counter(Path(file).parent.name for file in IMAGE_PATHS)
    print(dict(nbr_per_cat))

    categories = list(nbr_per_cat.keys())
    counts = list(nbr_per_cat.values())

    plt.figure(figsize=(9, 3))
    bars = plt.bar(categories, counts, width=0.5)

    plt.xlabel('Categories')
    plt.ylabel('Counts')
    plt.title('Number of Files per Category')
    plt.xticks(rotation=45, ha='right')

    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), str(count), 
                ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

get_distribution()

In [ ]:
def to_grayscale(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def resize_image(img, size):
    return cv2.resize(img, (size,size))

In [ ]:
def print_score(clf, x_train, y_train, x_test, y_test, train=True):
    if train:
        y_prediction = clf.predict(x_train)
        clf_report = classification_report(y_train, y_prediction)
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, y_prediction) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, y_prediction)}\n")
        
    elif train==False:
        y_prediction = clf.predict(x_test)
        clf_report = classification_report(y_test, y_prediction)
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, y_prediction) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, y_prediction)}\n")

In [ ]:
def extract_hog_features():
    hog_data = []
    hog_labels = []

    for p in tqdm(IMAGE_PATHS):
        im = to_grayscale(str(p))
        im = resize_image(im, 128)
        
        # extract hog feature descriptor
        fd1 = hog(im, orientations=7, pixels_per_cell=(8, 8),
                  cells_per_block=(4, 4), 
                  block_norm='L2-Hys',
                  transform_sqrt=False,
                  feature_vector=True)

        label = Path(p).parent.name
        hog_labels.append(label)
        hog_data.append(fd1)

    hog_data = np.array(hog_data)
    hog_labels = np.array(hog_labels)
    print("[INFO] Number of features ...", str(hog_data.shape[1]))
    print("[INFO] Number of labels ...", str(hog_labels.shape[0]))
    return hog_data, hog_labels

def compute_hog_feature_size(image_shape, orientations, pixels_per_cell, cells_per_block):
    height, width = image_shape, image_shape
    num_cells_x = width // pixels_per_cell
    num_cells_y = height // pixels_per_cell
    num_blocks_x = num_cells_x - cells_per_block + 1
    num_blocks_y = num_cells_y - cells_per_block + 1
    features_per_block = cells_per_block * cells_per_block * orientations
    total_features = num_blocks_x * num_blocks_y * features_per_block
    print("[INFO] Size of HOG feature vector ...", total_features)

# Parameters
orientations = 7
pixels_per_cell = 8
cells_per_block = 4
image_shape = 64

compute_hog_feature_size(image_shape, orientations, pixels_per_cell, cells_per_block)

In [ ]:
fig = plt.figure(figsize=(20,20))

for i in range(10):

    fig.add_subplot(1, 10, i + 1)
    plt.imshow(np.array(cv2.imread(str(IMAGE_PATHS[i]))), cmap='gray')
    label = Path(IMAGE_PATHS[i]).parent.name
    plt.title(label)

plt.show()

In [ ]:
hog_data, hog_labels = extract_hog_features()

x_train, x_test, y_train, y_test = train_test_split(hog_data, hog_labels, test_size=0.2, shuffle=True, stratify=hog_labels, random_state=42)
print("[INFO] Number of images used in training ...", x_train.shape[0])
print("[INFO] Number of images used in testing ...", x_test.shape[0])

classifier = SVC()
parameters = {'gamma': [0.1, 0.01, 0.001], 'C': [1, 10, 100, 1000]}

grid_search = GridSearchCV(classifier, parameters, n_jobs=-1)
grid_search.fit(x_train, y_train)

best_estimator_hog = grid_search.best_estimator_
print("[INFO] Best params ...", grid_search.best_params_)

pickle.dump(best_estimator_hog, open('/project/volume/models/EmoReact/hog_model.p', 'wb'))

print_score(best_estimator_hog, x_train, y_train, x_test, y_test, train=True)
print_score(best_estimator_hog, x_train, y_train, x_test, y_test, train=False)

In [180]:
def show_image(img):
    plt.imshow(img, cmap='gray')
    plt.show()

In [184]:
def read_mp_cv2_image(image_path):
    return mp.Image.create_from_file(str(image_path)), cv2.imread(str(image_path))

In [201]:
def crop_faces():
    for file in IMAGE_PATHS:
        img_mp, img_cv2 = read_mp_cv2_image(file)
        detection_results = detector_detect.detect(img_mp)
        file_stem = str(file.stem)
        file_parent = file.parent

        for d_idx, d in enumerate(detection_results.detections):
            bbox = d.bounding_box
            origin_x, origin_y, width, height = bbox.origin_x, bbox.origin_y, bbox.width, bbox.height
            face = img_cv2[origin_y:origin_y + height, origin_x:origin_x + width]

            frame_file_name = f"{file_stem}_face_{d_idx}.jpg"
            frame_path = file_parent / frame_file_name

            if not frame_path.exists():
                cv2.imwrite(str(frame_path), face)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

train_gen = keras.preprocessing.image.ImageDataGenerator()
valid_gen = keras.preprocessing.image.ImageDataGenerator()
test_gen = keras.preprocessing.image.ImageDataGenerator()